# Twitter API - get tweets with specific ID

### Install the Twython library

In [ ]:
# !pip install Twython

### import required libraries

In [ ]:
from twython import Twython
import pandas as pd

### Load IDs

In [ ]:
# BASE = 'D:\\ResearchDataGtx1060\\SentimentData\\Racism\\'
# fin = 'NAACL_SRW_2016_IDs.csv'
# fout = 'NAACL_SRW_2016_Tweets.csv'

BASE = 'D:\\ResearchDataGtx1060\\MisInformation\\'
fin = 'Dataset-I.txt'
fout = 'Dataset-I_Tweets.csv'

In [ ]:
df_in = pd.read_csv(BASE+fin, sep='\t', dtype=str)
#df_in = pd.read_csv(BASE+fin, header=None)
df_in

In [ ]:
df_in.groupby('class').count()

### Define API access information

In [ ]:
df_key = pd.read_csv('TwitterKyes.key', sep=':', header=None)
df_key.columns = ['Name', 'Val']
df_key

In [ ]:
# Generate your own at https://apps.twitter.com/app

CONSUMER_KEY = df_key.loc[0, 'Val'] # "<Consumer Key (API key)>"
CONSUMER_SECRET = df_key.loc[1, 'Val'] # "<Consumer Secret (API Secret)>"
OAUTH_TOKEN = df_key.loc[3, 'Val'] # "<application key (Access Token)>"
OAUTH_TOKEN_SECRET = df_key.loc[4, 'Val'] # "<application secret (Access Token Secret)>"

### Creat the API object

In [ ]:
twitter = Twython(CONSUMER_KEY, 
                  CONSUMER_SECRET,
                  OAUTH_TOKEN, 
                  OAUTH_TOKEN_SECRET)

### Test the API

In [ ]:
# tweet = twitter.show_status(id='1270081492908216320')
# print(tweet['text'])

In [ ]:
df_in.loc[0, 'id']

### Obtain the tweets for the IDs

In [ ]:
tweet_list = []
count = 0
for idx in df_in.index:
    try:
        tweet = twitter.show_status(id = df_in.loc[idx, 'id'])
        tweet_list.append(tweet)
        if count%100==0:
            print(tweet['text'])
        count += 1
    except:
        continue

In [ ]:
len(tweet_list)

In [ ]:
tweet_list

In [ ]:
len(df_in)

In [ ]:
def get_tweet(tid):
    for tweet in tweet_list:
        if tweet['id_str']==tid:
            return tweet
    return None

row_list = []
for idx in df_in.index:
    tweet = get_tweet(df_in.loc[idx, 'id'])
    if tweet!=None:
        row = {}
        row['id'] = df_in.loc[idx, 'id']
        row['text'] = tweet['text']
        row['label'] = df_in.loc[idx, 'class']
        row_list.append(row)

In [ ]:
len(row_list)

In [ ]:
df_out = pd.DataFrame(row_list)
df_out.head(10)

In [ ]:
df_out.groupby('label').count()

### Store the tweets

In [ ]:
#tweet_list
import json

with open(BASE+'Dataset-I_TweetsMeta.json', 'w') as FO:
    json.dump(tweet_list, FO)

In [ ]:
df_out.to_csv(BASE+fout, index=None)